In [1]:
from datetime import date, datetime, timedelta
from pytz import timezone
import pytz
utc = pytz.utc
# utc.zone
brussels = timezone('Europe/Brussels')
# brussels.zone
today = datetime.now(brussels).date()
# date.today()
today - timedelta(days=10)


datetime.date(2021, 11, 5)

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [3]:
from datetime import datetime, date
import pandas as pd
import numpy as np

In [4]:
from app.utils import get_db_engine

In [5]:
from app.etl.pipeline import Pipeline
from app.etl.pipeline import Transformer
from app.models import models
from app.models.metadata import ETL_Metadata

In [58]:
pl = {
  "source": "https://statbel.fgov.be/sites/default/files/files/opendata/bevolking%20naar%20woonplaats%2C%20nationaliteit%20burgelijke%20staat%20%2C%20leeftijd%20en%20geslacht/TF_SOC_POP_STRUCT_2021.zip",
  "model": "DemographicsByNISCodeAndCategory",
  "tranforms": [
    {
        "type": "drop_columns",
        "data": {
            "columns": [
                "TX_DESCR_NL",
                "TX_DESCR_FR",
                "CD_DSTR_REFNIS",
                "TX_ADM_DSTR_DESCR_NL",
                "TX_ADM_DSTR_DESCR_FR",
                "CD_PROV_REFNIS",
                "TX_PROV_DESCR_NL",
                "TX_PROV_DESCR_FR",
                "CD_RGN_REFNIS",
                "TX_RGN_DESCR_NL",
                "TX_RGN_DESCR_FR"
            ]
        }
    },
    {
        "type": "rename_columns",
        "data": {
            "columns": {
                "CD_REFNIS": "nis",
                "CD_SEX": "sex",
                "CD_NATLTY": "nationality_code",
                "TX_NATLTY_NL": "nationality_text_nl",
                "TX_NATLTY_FR": "nationality_text_fr",
                "CD_CIV_STS": "marital_status_code",
                "TX_CIV_STS_NL": "marital_status_text_nl",
                "TX_CIV_STS_FR": "marital_status_text_fr",
                "CD_AGE": "age",
                "MS_POPULATION": "population",
            }
        }
    },
    {
        "type": "add_column",
        "data": {
            "column": "year",
            "static_value": {
                "type": "integer",
                "value": 2021
            }
        }
    }
  ]
}

In [59]:
pipeline = Pipeline(
    data_class=getattr(models, pl["model"]),
    path=pl["source"],
    transformer=Transformer(pl["tranforms"])
)

In [60]:
data_frame = pipeline.extract()

In [61]:
data_frame = pipeline.transform(data_frame)

In [55]:
data_frame

,nis,sex,nationality_code,nationality_text_nl,nationality_text_fr,marital_status_code,marital_status_text_nl,marital_status_text_fr,age,population,year
0,11001,M,BEL,Belgen,Belges,4,Gescheiden,Divorcé,85,2,2021
1,11001,F,BEL,Belgen,Belges,4,Gescheiden,Divorcé,54,21,2021
2,11001,F,BEL,Belgen,Belges,4,Gescheiden,Divorcé,61,14,2021
3,11001,F,BEL,Belgen,Belges,4,Gescheiden,Divorcé,64,21,2021
4,11001,M,BEL,Belgen,Belges,4,Gescheiden,Divorcé,76,5,2021
...,...,...,...,...,...,...,...,...,...,...,...
467022,93090,M,BEL,Belgen,Belges,2,Gehuwd,Marié,61,21,2021
467023,93090,F,BEL,Belgen,Belges,2,Gehuwd,Marié,57,20,2021
467024,93090,F,BEL,Belgen,Belges,2,Gehuwd,Marié,53,22,2021
467025,93090,M,BEL,Belgen,Belges,2,Gehuwd,Marié,68,11,2021


In [66]:
list = [
    pipeline.data_class(**kwargs) for kwargs in data_frame.to_dict(orient="records")
]

In [67]:
list

 ...]

In [13]:
data_frame = pipeline.handle_metadata(data_frame)

In [25]:
data_frame

,nis,sex,nationality_code,nationality_text_nl,nationality_text_fr,marital_status_code,marital_status_text_nl,marital_status_text_fr,age,population,year
0,11001,M,BEL,Belgen,Belges,4,Gescheiden,Divorcé,85,2,2021
1,11001,F,BEL,Belgen,Belges,4,Gescheiden,Divorcé,54,21,2021
2,11001,F,BEL,Belgen,Belges,4,Gescheiden,Divorcé,61,14,2021
3,11001,F,BEL,Belgen,Belges,4,Gescheiden,Divorcé,64,21,2021
4,11001,M,BEL,Belgen,Belges,4,Gescheiden,Divorcé,76,5,2021
...,...,...,...,...,...,...,...,...,...,...,...
467022,93090,M,BEL,Belgen,Belges,2,Gehuwd,Marié,61,21,2021
467023,93090,F,BEL,Belgen,Belges,2,Gehuwd,Marié,57,20,2021
467024,93090,F,BEL,Belgen,Belges,2,Gehuwd,Marié,53,22,2021
467025,93090,M,BEL,Belgen,Belges,2,Gehuwd,Marié,68,11,2021


In [65]:
data_frame = data_frame.groupby([
    'year', 'nis', 'sex',
    'nationality_code', 'nationality_text_nl', 'nationality_text_fr',
    'marital_status_code',
    'marital_status_text_nl',
    'marital_status_text_fr',
    'age'
]).sum().reset_index()
data_frame

,year,nis,sex,nationality_code,nationality_text_nl,nationality_text_fr,marital_status_code,marital_status_text_nl,marital_status_text_fr,age,population
0,2021,11001,F,BEL,Belgen,Belges,1,Ongehuwd,Célibataire,0,42
1,2021,11001,F,BEL,Belgen,Belges,1,Ongehuwd,Célibataire,1,55
2,2021,11001,F,BEL,Belgen,Belges,1,Ongehuwd,Célibataire,2,65
3,2021,11001,F,BEL,Belgen,Belges,1,Ongehuwd,Célibataire,3,49
4,2021,11001,F,BEL,Belgen,Belges,1,Ongehuwd,Célibataire,4,54
...,...,...,...,...,...,...,...,...,...,...,...
466505,2021,93090,M,ETR,niet-Belgen,non-Belges,4,Gescheiden,Divorcé,74,1
466506,2021,93090,M,ETR,niet-Belgen,non-Belges,4,Gescheiden,Divorcé,75,1
466507,2021,93090,M,ETR,niet-Belgen,non-Belges,4,Gescheiden,Divorcé,78,1
466508,2021,93090,M,ETR,niet-Belgen,non-Belges,4,Gescheiden,Divorcé,79,1


In [31]:
data_frame.groupby(['year', 'nis', 'sex', 'nationality_code', 'marital_status_code', 'age'])['population'].transform('size')

0         1
1         1
2         1
3         1
4         1
         ..
467022    1
467023    1
467024    1
467025    1
467026    1
Name: population, Length: 467027, dtype: int64

In [32]:

data_frame[data_frame.groupby(['year', 'nis', 'sex', 'nationality_code', 'marital_status_code', 'age'])['nis'].transform('size') > 1]


,nis,sex,nationality_code,nationality_text_nl,nationality_text_fr,marital_status_code,marital_status_text_nl,marital_status_text_fr,age,population,year
573,11001,F,BEL,Belgen,Belges,3,Weduwstaat,Veuf,100,3,2021
592,11001,F,BEL,Belgen,Belges,3,Weduwstaat,Veuf,100,1,2021
1033,11002,F,BEL,Belgen,Belges,3,Weduwstaat,Veuf,100,51,2021
1048,11002,F,BEL,Belgen,Belges,3,Weduwstaat,Veuf,100,4,2021
1050,11002,F,BEL,Belgen,Belges,3,Weduwstaat,Veuf,100,5,2021
...,...,...,...,...,...,...,...,...,...,...,...
461386,92142,F,BEL,Belgen,Belges,3,Weduwstaat,Veuf,100,1,2021
461428,92142,F,BEL,Belgen,Belges,3,Weduwstaat,Veuf,100,1,2021
461455,92142,F,BEL,Belgen,Belges,3,Weduwstaat,Veuf,100,2,2021
466799,93090,F,BEL,Belgen,Belges,3,Weduwstaat,Veuf,100,1,2021


In [15]:
data_frame.describe()

,nis,marital_status_code,age,population,year
count,467027.000000,467027.000000,467027.000000,467027.000000,467027.0
mean,45726.380742,2.206316,53.053519,24.669319,2021.0
std,24016.466148,1.126056,23.578391,64.084580,0.0
min,11001.000000,1.000000,0.000000,1.000000,2021.0
25%,24028.000000,1.000000,36.000000,2.000000,2021.0
50%,44045.000000,2.000000,54.000000,7.000000,2021.0
75%,63035.000000,3.000000,72.000000,25.000000,2021.0
max,93090.000000,4.000000,100.000000,2870.000000,2021.0


In [16]:
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 467027 entries, 0 to 467026
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   nis                     467027 non-null  int64 
 1   sex                     467027 non-null  object
 2   nationality_code        467027 non-null  object
 3   nationality_text_nl     467027 non-null  object
 4   nationality_text_fr     467027 non-null  object
 5   marital_status_code     467027 non-null  int64 
 6   marital_status_text_nl  467027 non-null  object
 7   marital_status_text_fr  467027 non-null  object
 8   age                     467027 non-null  int64 
 9   population              467027 non-null  int64 
 10  year                    467027 non-null  int64 
dtypes: int64(5), object(6)
memory usage: 39.2+ MB


In [17]:
data_frame.isnull().sum()

nis                       0
sex                       0
nationality_code          0
nationality_text_nl       0
nationality_text_fr       0
marital_status_code       0
marital_status_text_nl    0
marital_status_text_fr    0
age                       0
population                0
year                      0
dtype: int64

In [18]:
data_frame.dropna(inplace=True)

In [19]:
data_frame.isnull().sum()

nis                       0
sex                       0
nationality_code          0
nationality_text_nl       0
nationality_text_fr       0
marital_status_code       0
marital_status_text_nl    0
marital_status_text_fr    0
age                       0
population                0
year                      0
dtype: int64

In [20]:
test = 2021
type(test)

int

In [21]:
starting_day_of_current_year = datetime.now().date().replace(month=1, day=1)    
ending_day_of_current_year = datetime.now().date().replace(month=12, day=31)
starting_day_of_current_year

datetime.date(2021, 1, 1)

In [22]:
frequency = "daily"
frequency is not "daily"

<>:2: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:2: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/var/folders/vh/2z356ng15yq3zcc_tlvwjlqw0000gn/T/ipykernel_16771/1665065483.py:2: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  frequency is not "daily"


False